# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [1]:
%reload_ext autoreload
%autoreload 2

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

from src.acnets.deep import ACNetsDataModule, MultiHeadModel

In [2]:
atlases = [
    'dosenbach2010',
    'dosenbach2007',
    'difumo_64_2mm',
    'difumo_128_2mm',
    'difumo_256_2mm',
    'difumo_512_2mm',
    'difumo_1024_2mm',
    'gordon2014_2mm',
    'cort-maxprob-thr25-2mm',
    'seitzman2018',
    'friedman2020',
]

kinds = [
    'correlation',
    'partial correlation',
    'tangent',
    'covariance',
    'precisions'
]

In [3]:
datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation')
datamodule.setup()

n_regions = datamodule.train[0][0].shape[1]
n_networks = datamodule.train[0][2].shape[1]
model = MultiHeadModel(n_regions, n_networks)

trainer = pl.Trainer(accelerator='auto', max_epochs=10000,
                     log_every_n_steps=3, callbacks=[RichProgressBar()])

trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ train_accuracy │ MulticlassAccuracy │      0 │
│ 1 │ val_accuracy   │ MulticlassAccuracy │      0 │
│ 2 │ h1_encoder     │ LSTM               │  5.4 K │
│ 3 │ h1_output      │ Linear             │     18 │
│ 4 │ h2_head        │ Linear             │      2 │
│ 5 │ h3_encoder     │ LSTM               │    512 │
│ 6 │ h3_output      │ Linear             │     18 │
│ 7 │ h4_head        │ Sequential         │    222 │
│ 8 │ h5_head        │ Sequential         │    222 │
│ 9 │ h4_h5_output   │ Sequential         │     56 │
└───┴────────────────┴────────────────────┴────────┘

Trainable params: 6.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=10000` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/accuracy       │           0.75            │
│   test/dropout_accuracy   │     0.699999988079071     │
│         test/loss         │    0.5966799259185791     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.5966799259185791,
  'test/accuracy': 0.75,
  'test/dropout_accuracy': 0.699999988079071}]